In [1]:
from IPython.core.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = False

display(HTML("<style>.container { width:95% !important; }</style>"))

# Tasks 

- [x] Explain Simple and Optimized Implementations of the Algorithm Step by Step

# Imports

In [2]:
from drawdata import draw_scatter
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import math
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

In [3]:
np.random.seed(42)

# Create Dummy Data

In [32]:
# draw_scatter()

In [33]:
# df = pd.read_clipboard(sep=',')

In [11]:
Path.cwd()

WindowsPath('D:/amit/ml_indepth/perceptron')

In [35]:
# df.to_csv('dummy__perceptron.csv', index=None)

In [36]:
df = pd.read_csv('dummy__perceptron.csv')

In [37]:
df.z = df.z.apply(lambda x: 1 if x=='a' else -1)

In [38]:
df.z.value_counts()

 1    393
-1    277
Name: z, dtype: int64

# Data Prep

In [39]:
X_train_dummy = df[['x', 'y']].values
y_train_dummy = df.z.values

In [40]:
X_train_dummy.shape, y_train_dummy.shape

((670, 2), (670,))

In [41]:
X_train_dummy = np.hstack((X_train_dummy, np.ones((X_train_dummy.shape[0],1))))

In [42]:
X_train_dummy.shape

(670, 3)

In [43]:
X_train_dummy[0]

array([541.18323885, 494.08355305,   1.        ])

In [44]:
y_train_dummy[0]

1

# Basic Perceptron Implementation

In [46]:
# initialize the w
w = np.zeros((1,3))

In [47]:
w.shape

(1, 3)

In [48]:
X_train_dummy[0].dot(w.T)

array([0.])

In [49]:
dfx = pd.DataFrame(X_train_dummy, columns=['x', 'y', 'b'])
dfx['class'] = y_train_dummy

In [50]:
fig_unscaled = px.scatter(data_frame=dfx, x='x', y='y', color='class')

In [51]:
std_scaler = StandardScaler()
normalizer = Normalizer()
mm_scaler = MinMaxScaler()

In [52]:
X_train_dummy_std_scaled = std_scaler.fit_transform(X_train_dummy)
X_train_dummy_normalized = normalizer.fit_transform(X_train_dummy)
X_train_dummy_mm_scaled = mm_scaler.fit_transform(X_train_dummy)

In [53]:
dfx_std_scaled = pd.DataFrame(X_train_dummy_std_scaled,
                              columns=['x', 'y', 'b'])
dfx_std_scaled['class'] = y_train_dummy
fig_std_scaled = px.scatter(data_frame=dfx_std_scaled,
                            x='x',
                            y='y',
                            color='class')

dfx_normalized = pd.DataFrame(X_train_dummy_normalized,
                              columns=['x', 'y', 'b'])
dfx_normalized['class'] = y_train_dummy
fig_normalized = px.scatter(data_frame=dfx_normalized,
                            x='x',
                            y='y',
                            color='class')

dfx_mm_scaled = pd.DataFrame(X_train_dummy_mm_scaled, columns=['x', 'y', 'b'])
dfx_mm_scaled['class'] = y_train_dummy
fig_mm_scaled = px.scatter(data_frame=dfx_mm_scaled,
                           x='x',
                           y='y',
                           color='class')

In [54]:
fig_unscaled

In [55]:
fig_std_scaled

In [56]:
fig_normalized

In [57]:
fig_mm_scaled

In [58]:
while True:
    m = 0
    for x, y in zip(X_train_dummy, y_train_dummy):
        if y * (x.reshape(1,3).dot(w.T)) <= 0:
            w += y*x.reshape(1,3)
            m += 1
    if m == 0 :
        break


In [59]:
w

array([[ 3.44547961e+00,  2.96368580e+01, -9.54600000e+03]])

In [60]:
def predict(x):
    if x.reshape(1,3).dot(w.T) > 0:
        return 1
    else:
        return -1
    

In [61]:
predict(X_train_dummy[10]), y_train_dummy[10]

(1, 1)

# Perceptron Other Implementations Using Step Transfer Function

In [128]:
class Model():
    def __init__(self):
        self.num_input_features = None
        
    def fit(self, X, y):
        """ Fit the model.
        Args:
            X: A compressed sparse row matrix of floats with shape
                [num_examples, num_features].
            y: A dense array of ints with shape [num_examples].
        """
        raise NotImplementedError()
    
    def predict(self, X):
        """ Predict.
        Args:
            X: A compressed sparse row matrix of floats with shape
                [num_examples, num_features].

        Returns:
            A dense array of ints with shape [num_examples].
        """
        raise NotImplementedError()

In [129]:
class CustomPerceptron1(Model):
    
    def __init__(self, max_iter, n):
        super().__init__()
        self.max_iter = max_iter
        self.n = n 
        
        
    def fit(self, X, y):
        self.ne, self.nf = np.shape(X)
        self.w = np.zeros(self.nf)
        
#         for i in range(0, self.max_iter):
        while True:
            m = 0
            for e, l in zip(X,y):
                if l*np.dot(self.w, e) <= 0:
                    self.w += l * e
                    m += 1
            if m == 0:
                break
                    
    def predict(self, X):
        ne, nf = np.shape(X)
        if nf != self.nf:
            X = X[:, :self.nf]
        
        predicted_Y = np.empty([ne])
        
        for i in range(ne):
            if np.dot(self.w, X[i])>0:                           
                predicted_Y[i] = 1
            else:
                predicted_Y[i] = -1
            
        return predicted_Y
            
    

In [65]:
model = Perceptron(10000, 0.1)

In [66]:
%%time
model.fit(X_train_dummy_std_scaled, y_train_dummy)

Wall time: 7.16 ms


In [67]:
model.w

array([-0.13486593,  3.18068067,  0.        ])

In [68]:
model.predict(X_train_dummy_std_scaled[-20:])

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1.])

In [65]:
# Took too long to converge
# %%time 
# model.fit(X_train_dummy_mm_scaled, y_train_dummy)

KeyboardInterrupt: 

In [66]:
# model.w

array([-0.67654049, -0.44843048,  0.        ])

In [67]:
# model.predict(X_train_dummy_mm_scaled[-20:])

In [72]:
%%time 
model.fit(X_train_dummy_normalized, y_train_dummy)

Wall time: 2min 5s


In [73]:
model.w

array([ 2.96666055e-02,  1.14696090e+00, -2.88355625e+02])

In [74]:
model.predict(X_train_dummy_normalized[-20:])

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1.])

In [32]:
%%time
model.fit(X_train_dummy, y_train_dummy)

Wall time: 21.9 s


In [33]:
model.w

array([-1.37993956e+00,  1.84097266e+02, -4.43150000e+04])

In [34]:
model.predict(X_train_dummy[-20:])

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1.])

In [35]:
y_train_dummy[-20:]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1], dtype=int64)

In [240]:
model.w.shape, X_train_dummy.shape

((3,), (686, 3))

In [37]:
np.linalg.norm(X_train_dummy[0])

769.5062505018591

In [38]:
np.linalg.norm(X_train_dummy_scaled[0])

1.835660334057529

# Perceptron using Stochastic Gradient Descent

## 1. Make Predictions

In [135]:
trial_set = np.concatenate((X_train_dummy_std_scaled[-10:,:-1], y_train_dummy[-10:].reshape(-1,1)), axis=1)
trial_set

array([[-1.48191319, -0.56077897, -1.        ],
       [-0.98930089, -0.56096287, -1.        ],
       [-0.79511501, -0.3625388 , -1.        ],
       [-1.28252626, -0.24761745, -1.        ],
       [-1.1359904 , -0.45413882, -1.        ],
       [-0.74719578, -0.45130754, -1.        ],
       [-0.31248589, -0.66825231, -1.        ],
       [ 0.38781868, -0.22366354, -1.        ],
       [ 0.18600259, -0.29907397, -1.        ],
       [ 0.12436777, -0.66394631, -1.        ]])

In [136]:
def predict(x, weights):
    activation = weights[0]
    for i in range(len(x)-1):
        activation += weights[i+1] * x[i]
    return 1. if activation>=0.0 else 0.0

In [82]:
weights = [-0.1, 0.20653640140000007, -0.23418117710000003]
# setup is bias first [bias, w1, w2]
# activation = (w1*x1) + (w2*x2) + bias

In [139]:
for i in range(len(trial_set)-1):
    prediction = predict(trial_set[i], weights)
    print("Expected=%d, Predicted=%d" % (trial_set[i][-1], prediction))

Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1
Expected=-1, Predicted=1


## 2. Training Network Weights

In [143]:
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row[:-1], weights)
            error = row[-1] - prediction 
            sum_error += error**2
            weights[0] =  weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i+1] = weights[i+1] + l_rate * error * row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return weights

In [144]:
l_rate = 0.1
n_epoch = 5
weights = train_weights(trial_set, l_rate, n_epoch)
weights

>epoch=0, lrate=0.100, error=13.000
>epoch=1, lrate=0.100, error=10.000
>epoch=2, lrate=0.100, error=10.000
>epoch=3, lrate=0.100, error=10.000
>epoch=4, lrate=0.100, error=10.000


[-5.099999999999998, 3.171360511923467, 2.3022181866516562]

## Testing on Sonar Dataset

In [151]:
from random import seed
from random import randrange
from csv import reader

In [152]:
# load csv file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [159]:
# convert string to float
def str_column_to_float(dataset, columns):
    for row in dataset:
        row[columns] = float(row[columns].strip())

In [154]:
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [157]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
 
# Make a prediction with weights
def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0
 
# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights
 
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)
 

In [162]:
# Test the Perceptron algorithm on the sonar dataset
seed(1)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [76.81159420289855, 69.56521739130434, 72.46376811594203]
Mean Accuracy: 72.947%


# Perceptron Implementation 2

In [110]:
class CustomPerceptron(object):
    def __init__(self, n_inputs, threshold=1000, learning_rate=0.1):
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.weights = np.zeros(n_inputs+1)
        
    def predict(self, inputs):
        summationn = np.dot(inputs, self.weights[1:]) + self.weights[0]
        if summationn > 0:
            activation = 1
        else:
            activation = 0
        return activation
    
    def train(self, training_inputs, labels):
        for _ in range(self.threshold):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                self.weights[1:] += self.learning_rate * (label-prediction) * inputs
                self.weights[0] += self.learning_rate * (label-prediction)


In [70]:
y_train_dummy = [1 if i == 1 else 0 for i in y_train_dummy]

In [71]:
model = CustomPerceptron(n_inputs=2, threshold=10000, learning_rate=0.1)

In [72]:
model.train(X_train_dummy_std_scaled[:,:-1], y_train_dummy)

In [73]:
model.weights

array([0.1       , 0.0923207 , 0.28477031])

In [74]:
X_train_dummy_std_scaled[:,:][0]

array([1.07833151, 1.29151418, 0.        ])

In [75]:
model.predict(X_train_dummy_std_scaled[:,:-1][-1])

0

# SkLearn Implementation and Comparision

In [4]:
from sklearn.linear_model import Perceptron
import sklearn.datasets as datasets

In [5]:
Path.cwd()

WindowsPath('D:/amit/ml_indepth/perceptron')

In [6]:
data_path = Path('D:/amit/ml_indepth/datasets/')

In [7]:
list(data_path.glob('*'))

[WindowsPath('D:/amit/ml_indepth/datasets/diabetes.xlsx'),
 WindowsPath('D:/amit/ml_indepth/datasets/sonar.all-data.csv')]

In [8]:
df = pd.read_excel(data_path/'diabetes.xlsx')

In [9]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
103,1,81,72,18,40,26.6,0.283,24,0
104,2,85,65,0,0,39.6,0.930,27,0
105,1,126,56,29,152,28.7,0.801,21,0
106,1,96,122,0,0,22.4,0.207,27,0


In [104]:
X = df.iloc[:,:-1].values
y = df.Outcome.values

In [108]:
model = Perceptron(random_state=42)
model.fit(X,y)

Perceptron(random_state=42)

In [109]:
print("%0.3f" % model.score(X, y))

0.676


In [119]:
model1 = CustomPerceptron(n_inputs=X.shape[-1], threshold=100000, learning_rate=0.1)

In [120]:
model1.train(X, y)

In [121]:
model1.weights

array([-8285.70000001,   161.2       ,    22.9       ,   -32.3       ,
          15.3       ,   -11.4       ,    55.81      ,   638.0266    ,
          43.2       ])

In [122]:
model.coef_

array([[ 142.   , -119.   , -278.   , -237.   ,  203.   ,  100.3  ,
          10.677,  208.   ]])

In [123]:
predictions = []
for x in X:
    predictions.append(model1.predict(x))

In [125]:
sum([True if x==y else False for x,y in zip(y, predictions)])/len(y)

0.6481481481481481

In [127]:
df.Outcome.value_counts()/df.shape[0]

0    0.648148
1    0.351852
Name: Outcome, dtype: float64

In [130]:
model2 = CustomPerceptron1(100000, 0.1)

In [131]:
# Runs forever so our linear separation assumtion fails
model2.fit(X, y)

KeyboardInterrupt: 

In [2]:
# df

# Real World Use Case 2

In [3]:
import numpy as np
import pandas as pd 
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron

In [4]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=10, n_redundant=0, n_classes=2, random_state=42)

In [7]:
model = Perceptron()

In [8]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

In [9]:
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [11]:
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Mean Accuracy: 0.763 (0.060)


In [12]:
scores

array([0.86, 0.84, 0.71, 0.74, 0.58, 0.77, 0.79, 0.74, 0.8 , 0.84, 0.8 ,
       0.75, 0.71, 0.75, 0.8 , 0.76, 0.79, 0.72, 0.68, 0.8 , 0.78, 0.79,
       0.81, 0.79, 0.66, 0.72, 0.77, 0.7 , 0.86, 0.78])